# Ice Diagnostics

In [1]:
from glob import glob
import pandas as pd
import os
import xarray as xr
import cartopy
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature
%matplotlib inline
from IPython.display import HTML

In [2]:
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [9]:
DataDir = '/g/data/hh5/tmp/cosima/mom01v5'
expt = 'KDS75'

In [10]:
expdir = os.path.join(DataDir, expt)
dataFileList = glob(os.path.join(expdir, 'output*/ice_month.nc'))
dataFileList.sort()

In [11]:
gridFileList = glob(os.path.join(expdir, 'output*/ocean_grid.nc'))
gridFileList.sort()

In [12]:
print(dataFileList, gridFileList)

[] []


In [6]:
dsx_grid = xr.open_dataset(gridFileList[0], decode_times=False)
area_t = dsx_grid.area_t

IndexError: list index out of range

In [ ]:
dsx_grid.area_t.plot()

From `diag_table`


Saved in `ice_month.nc`

CN, HI, HS
averaged globally over each month


In [ ]:
dsx = xr.open_mfdataset(dataFileList[:12], decode_times=False, concat_dim='time')

There is an outstanding issue with year outside of a given range.  Artificially shift the years to years since 2000.

In [ ]:
dsx.time.attrs['units'] = 'days since 2000-01-01 00:00:00'
dsx.average_T1.attrs['units'] = 'days since 2000-01-01 00:00:00'
dsx.average_T2.attrs['units'] = 'days since 2000-01-01 00:00:00'
dsx = xr.decode_cf(dsx)

Annual ice volume

In [ ]:
HI = dsx.sel(yt=slice(-90, -60)).HI
south_area_t = area_t.sel(yt_ocean=slice(-90,-60))
HI

In [ ]:
volume = south_area_t.values * HI
volume.name = 'Volume'

In [ ]:
volume.sum(dim='yt').sum(dim='xt').plot(marker='o')

In [ ]:
import matplotlib as mpl
from matplotlib import rc

rc('animation', ffmpeg_path='/short/v45/jm0634/conda/envs/my_analysis3/bin/ffmpeg')
rc('animation', html='html5')

In [ ]:
plt.figure(figsize=[8,8])
ax1 = plt.subplot(1,1,1, projection=ccrs.SouthPolarStereo())
#ax1.set_extent([-180, 180, -90, -50], ccrs.PlateCarree())
ax1.add_feature(cartopy.feature.LAND)
#ax1.add_feature(cartopy.feature.OCEAN)
ax1.coastlines()
#ax1.stock_img()
ax1.gridlines()

quadmesh = dsx.HI.isel(time=0)\
            .sel(yt=slice(-90, -60))\
            .plot\
            .pcolormesh(ax=ax1, 
                        transform=ccrs.PlateCarree(),
                        vmax=2)

In [ ]:
import matplotlib.animation as animation

fig = plt.figure(figsize=[8,8])

ax1 = plt.subplot(1,1,1, projection=ccrs.SouthPolarStereo())
#ax1.set_extent([-180, 180, -90, -50], ccrs.PlateCarree())
ax1.add_feature(cartopy.feature.LAND)
#ax1.add_feature(cartopy.feature.OCEAN)
ax1.coastlines()
#ax1.stock_img()
ax1.gridlines()

quadmesh = dsx.HI.isel(time=0).sel(yt=slice(-90, 
                                 -60)).plot(ax=ax1, 
                                            transform=ccrs.PlateCarree(),
                                            vmax=2)
def animate(i):
    print(i)
    da = dsx.HI.isel(time=i).sel(yt=slice(-90,-60))
    quadmesh.set_array(da.to_masked_array().ravel())
    ax1.set_title('time = {}'.format(da.time.values))
    return quadmesh, ax1.title

def init_func():
    return quadmesh,